<a href="https://colab.research.google.com/github/yendruamarendra/Delta/blob/main/delta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TASK 2

# Install Pyspark


In [1]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 45 kB/s 
     |████████████████████████████████| 200 kB 49.4 MB/s 
     |████████████████████████████████| 199 kB 15.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=fa82c6d951a4cd57f6a6250bc23365358c04e4699d39336d4adb66cd4c648a78
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


Imports

In [3]:
from pyspark.sql import SparkSession
from pyspark.shell import spark
import pyspark.sql.functions as f
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from datetime import datetime

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.3.1
      /_/

Using Python version 3.8.16 (default, Dec  7 2022 01:12:13)
Spark context Web UI available at http://d805b4cbf3cc:4040
Spark context available as 'sc' (master = local[*], app id = local-1671654377091).
SparkSession available as 'spark'.


In [5]:
spark =  SparkSession.builder.appName("Delta_flights").getOrCreate()

# Read Flight leg Data as dataframe


In [31]:
df = spark.read.csv("/content/Delta/Dummy_Flight_Leg_Data.csv",header = True, inferSchema=True)

In [32]:
df.count()

244080

## Considering flighkey as primary filtering out records for null if there are any

In [33]:
df = df.filter(f.col("flightkey").isNotNull())

## Using window function for partition and order by

In [34]:
windowSpec = Window.partitionBy("flightkey").orderBy(f.col("flight_dt").desc(),f.col("lastupdt").desc())

## Creating Column row_number over windowSpec and selection first record to get recent status of flights

In [35]:
df_res = df.withColumn("row_number",row_number().over(windowSpec)).filter("row_number=1").drop("row_number")

## Displaying records for all the flights with most recent status

In [36]:
df_res.count()

244078

## USING SQL - TASK 1

In [37]:
df.createOrReplaceTempView("flight")

In [39]:
spark.sql("""SELECT * FROM ( 
  SELECT *, ROW_NUMBER() 
  OVER(PARTITION BY flightkey ORDER BY flight_dt desc,lastupdt DESC) AS row_number 
  FROM flight ) temp 
WHERE Row_number=1""").count()

244078